In [ ]:
%run ../prelude.py

## Create an adjacency matrix

In [ ]:
adjmat = Tensor.fromRandom(["S", "D"], [10, 10], (1.0, 0.3), 1, seed=10)
adjmat.setName("AdjMat")

displayTensor(adjmat)

## Create a frontier (type1)

In [ ]:
frontier1 = Tensor.fromRandom(["S"], [10], (0.4,), 1, seed=20)
frontier1.setName("Frontier1")

displayTensor(frontier1)

## Simple advance kernel (type1)

In [ ]:
next_frontier1 = Tensor(rank_ids=["S"])
next_frontier1.setName("Next_Frontier1")

adjmat_s = adjmat.getRoot()
frontier_s =  frontier1.getRoot()
next_frontier_s = next_frontier1.getRoot()

canvas = createCanvas(frontier1, adjmat, next_frontier1)

for s, (frontier_val, adjmat_d) in frontier_s & adjmat_s:
    for d, (next_frontier_ref, _) in next_frontier_s << adjmat_d:
        next_frontier_ref <<= 1
        addFrame(canvas, (s,), (s, d), (d,))
        
displayCanvas(canvas)

## Create a frontier (type2)

In [ ]:
frontier2 = Tensor.fromUncompressed("S", frontier1.getRoot().getCoords())
frontier2.setName("Frontier2")

displayTensor(frontier2)

In [ ]:
next_frontier2 = Tensor(rank_ids=["X"])
next_frontier1.setName("Next_Frontier2")

adjmat_s = adjmat.getRoot()

frontier_x =  frontier2.getRoot()
next_frontier_x = next_frontier2.getRoot()

canvas = createCanvas(frontier2, adjmat, next_frontier2)

i = 0

for x, frontier_sval in frontier_x:
    adjmat_d = adjmat_s.getPayload(frontier_sval, start_pos=0)
    for d, _ in adjmat_d:
        next_frontier_x.append(i, d)
        i += 1
        addFrame(canvas, (x, ), (frontier_sval, d), (i,))

(n, total) = adjmat_s.getSavedPosStats()
print(f"Average search distance = {total/n:4.2f}")

displayTensor(next_frontier2)
displayCanvas(canvas)

Optimize search in getPayload()

In [ ]:
next_frontier2 = Tensor(rank_ids=["X"])
next_frontier1.setName("Next_Frontier2")

adjmat_s = adjmat.getRoot()

frontier_x =  frontier2.getRoot()
next_frontier_x = next_frontier2.getRoot()

canvas = createCanvas(frontier2, adjmat, next_frontier2)

i = 0
last_pos = 0

for x, frontier_sval in frontier_x:
    adjmat_d = adjmat_s.getPayload(frontier_sval, start_pos=last_pos)
    last_pos = adjmat_s.getSavedPos()
    
    for d, _ in adjmat_d:
        next_frontier_x.append(i, d)
        i += 1
        addFrame(canvas, (x, ), (frontier_sval, d), (i,))

(n, total) = adjmat_s.getSavedPosStats()
print(f"Average search distance = {total/n:4.2f}")

displayTensor(next_frontier2)
displayCanvas(canvas)